# Find correlations in your daily eating habits

In [8]:
#@title Computer Setup
#@markdown Press the play button to the right to continue. 

import os
import pandas as pd
import numpy as np
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'mental_health_food')
drive.mount(MOUNTPOINT) #, force_remount=True)

datastore_fp = os.path.join(DATADIR, "datastore.csv") # I only do this for hackathons i promise

def get_df():
    return pd.DataFrame({"Meal":[], "Mood":[], "Time_of_Day":[], "Time_Recorded":[]})

if not os.path.exists(datastore_fp):
    os.makedirs(DATADIR, exist_ok=True)
    datastore = get_df()
    datastore.to_csv(datastore_fp, index=False)
else:
    datastore = pd.read_csv(datastore_fp)

cell_ran1 = True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
#@title 1. Add to your log
#@markdown Press the play button to the right to continue. Please include any meals or activities you think may be related to your mental health.
try:
    assert cell_ran1
except:
    raise Exception("Please run setup cell first. (cell 1)")

if not os.path.exists(datastore_fp):
    os.makedirs(DATADIR, exist_ok=True)
    datastore = get_df()
    datastore.to_csv(datastore_fp, index=False)
    
class DailyRecord:

    def __getitem__(self, key):
        return getattr(self, key)

    def __setitem__(self, key, val):
        setattr(self, key, val)

    def __contains__(self, key):
        return hasattr(self, key)
    
    #@markdown #### What meal did you have?
    Meal = "cereal" #@param { type: "string" }
    
    #@markdown #### What is your mood after your meal?
    Mood = "8" #@param ["N/A", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

    #@markdown #### When did you eat your meal?
    Time_of_Day = "Morning" #@param ["N/A", "Morning", "Noon", "Night"]


record = DailyRecord()
datastore = datastore.append({"Meal":record.Meal, "Mood":record.Mood, "Time_of_Day":record.Time_of_Day, "Time_Recorded":datetime.datetime.now()}, ignore_index = True)
datastore.to_csv(datastore_fp, index=False)

print(f"Data recorded at {datetime.datetime.now()}")

Data recorded at 2022-02-20 11:17:33.341656


In [16]:
#@markdown To clear your logs, type "True" and click the play button to the left
reset = "False" #@param ["True", "False"]
if reset == "True":
    datastore = get_df()
    datastore.to_csv(datastore_fp, index=False)

In [17]:
#@markdown 5 most recent meals
datastore.head(5)

,Meal,Mood,Time_of_Day,Time_Recorded
0,pizza,2,Morning,2022-02-20 10:43:49.237929
1,cereal,6,Night,2022-02-20 10:44:00.481336
2,mouthwash,6,Noon,2022-02-20 10:44:09.327923
3,mouthwash,1,Noon,2022-02-20 10:44:17.837231
4,pizza,3,Noon,2022-02-20 10:44:25.287492


In [21]:
#@title 2. Get Prediction for what could possibly lead to bad mood
try:
    assert cell_ran1
except:
    raise Exception("Please run setup cell first. (cell 1)")

# Preprocess
datastore_ = datastore.copy()
datastore_ = datastore_.drop(columns="Time_Recorded")
datastore_["Mood"] = pd.to_numeric(datastore["Mood"])
datastore_["Time_of_Day"] = datastore_["Time_of_Day"].apply(lambda x: {"N/A":-1, "Morning":0, "Noon":1, "Night":2}[x])

enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(datastore_.drop(columns=["Mood"]))
y = datastore["Mood"].to_numpy()
X.toarray()

# Train
machine_learning_good = True
realism_good = not machine_learning_good

if machine_learning_good:
    model = MLPRegressor(random_state=1, max_iter=500)
elif realism_good:
    model = LinearRegression()
else:
    raise NotImplementedError()
  
reg = model.fit(X, y)
reg.score(X, y)

# Predict
worst_idx = np.argmin(reg.predict(X))
worst_meal = datastore_.iloc[worst_idx]["Meal"]
print(f"We suspect that {worst_meal} may be negatively affecting your mood. Consider consulting your doctor for more advice.")

We suspect that pizza may be negatively affecting your mood. Consider consulting your doctor for more advice.
